# Source Tracking Simulation (Pure Disk)

**Purpose**: Track where Trojans originate from by starting with a pure protoplanetary disk.

This simulation:
1. Starts with a broad disk (2-35 AU) of test particles
2. Evolves with Jupiter at a fixed position (no migration)
3. Identifies which initial regions supply particles to L4/L5

The results help validate particle injection for migration simulations.

## 1. Setup and Imports

In [1]:
import sys
sys.path.insert(0, '.')

import numpy as np
import matplotlib.pyplot as plt
from amuse.units import units

from simulation import (
    SimulationConfig,
    run_simulation,
    load_checkpoint,
    plot_trojan_evolution,
    plot_snapshot,
    create_animation,
    compute_population_summary,
    classify_planetesimals
)

print('Simulation module loaded successfully!')

Simulation module loaded successfully!


/Users/zoutongshen/miniforge3/envs/Amuse-env/lib/python3.12/site-packages/amuse/support/options.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## 2. Configuration

Source tracking uses a broader disk than burn-in simulations.

In [2]:
# === SOURCE TRACKING CONFIGURATION ===

# Duration in thousands of years
DURATION_KYR = 100

# Number of planetesimals (more for better statistics)
N_PLANETESIMALS = 10000

# Disk extent (AU) - broader than Trojan region
DISK_A_MIN = 2.0   # Inner edge
DISK_A_MAX = 35.0  # Outer edge

# Number of snapshots
N_SNAPSHOTS = 200

# Create configuration
config = SimulationConfig.source_tracking(
    duration_kyr=DURATION_KYR,
    n_planetesimals=N_PLANETESIMALS,
    n_snapshots=N_SNAPSHOTS,
    disk_a_min=DISK_A_MIN,
    disk_a_max=DISK_A_MAX
)

print(f'Configuration: {config.name}')
print(f'  Duration: {config.end_time:.0f} years')
print(f'  Disk: {DISK_A_MIN} - {DISK_A_MAX} AU')
print(f'  Planetesimals: {config.n_planetesimals}')
print(f'  Output: {config.output_file}')

Configuration: jupiter_source_tracking_100kyr
  Duration: 100000 years
  Disk: 2.0 - 35.0 AU
  Planetesimals: 10000
  Output: jupiter_source_tracking_100kyr.pkl


## 3. Run Simulation

In [ ]:
# Run the simulation
results = run_simulation(config, verbose=True)

## 4. Standard Analysis

In [ ]:
# Extract results
times = results['times']
l4_counts = results['l4_counts']
l5_counts = results['l5_counts']
snapshots_massive = results['snapshots_massive']
snapshots_planetesimals = results['snapshots_planetesimals']

# Plot evolution
import os
os.makedirs('results/png', exist_ok=True)

fig = plot_trojan_evolution(
    times, l4_counts, l5_counts,
    title=f'Trojan Evolution ({config.name})'
)
fig.savefig(f'results/png/{config.name}_evolution.png', dpi=150)

In [ ]:
# Population summary
summary = compute_population_summary(times, l4_counts, l5_counts)

print('=' * 60)
print('POPULATION SUMMARY')
print('=' * 60)
print(f"Initial: L4={summary['initial_l4']}, L5={summary['initial_l5']}")
print(f"Final:   L4={summary['final_l4']}, L5={summary['final_l5']}")
print(f"L4/L5 ratio slope: {summary['ratio_slope']:.3e} yr⁻¹")
print('=' * 60)

## 5. Source Region Analysis

This is the key analysis: where did the final Trojans come from?

In [ ]:
# Get initial and final particle positions
initial_planetes = snapshots_planetesimals[0]
final_planetes = snapshots_planetesimals[-1]
final_massive = snapshots_massive[-1]

# Classify final particles
classification = classify_planetesimals(final_massive, final_planetes)

print(f"Final classification:")
print(f"  L4 Trojans: {len(classification['L4'])}")
print(f"  L5 Trojans: {len(classification['L5'])}")
print(f"  Other co-orbital: {len(classification['coorbital_other'])}")
print(f"  Inner region: {len(classification['inner'])}")
print(f"  Outer region: {len(classification['outer'])}")

In [ ]:
# Track back: where did L4 and L5 Trojans originate?
def get_initial_radii(indices, initial_particles):
    """Get initial radii for particles with given indices."""
    radii = []
    for idx in indices:
        p = initial_particles[idx]
        r = p.position.length().value_in(units.AU)
        radii.append(r)
    return np.array(radii)

# Get initial radii of final Trojans
l4_initial_radii = get_initial_radii(classification['L4'], initial_planetes)
l5_initial_radii = get_initial_radii(classification['L5'], initial_planetes)

# Plot source regions
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# All initial particles
all_initial_radii = [p.position.length().value_in(units.AU) for p in initial_planetes]
axes[0].hist(all_initial_radii, bins=50, range=(0, 40), alpha=0.7, color='gray')
axes[0].axvline(5.2, color='brown', linestyle='--', linewidth=2, label='Jupiter')
axes[0].set_xlabel('Initial Distance (AU)')
axes[0].set_ylabel('Count')
axes[0].set_title('All Initial Particles')
axes[0].legend()

# L4 sources
if len(l4_initial_radii) > 0:
    axes[1].hist(l4_initial_radii, bins=30, range=(0, 40), alpha=0.7, color='blue')
    axes[1].axvline(5.2, color='brown', linestyle='--', linewidth=2)
    axes[1].axvline(np.median(l4_initial_radii), color='blue', linestyle='-', 
                    linewidth=2, label=f'Median: {np.median(l4_initial_radii):.1f} AU')
axes[1].set_xlabel('Initial Distance (AU)')
axes[1].set_ylabel('Count')
axes[1].set_title(f'L4 Trojan Sources (n={len(l4_initial_radii)})')
axes[1].legend()

# L5 sources
if len(l5_initial_radii) > 0:
    axes[2].hist(l5_initial_radii, bins=30, range=(0, 40), alpha=0.7, color='red')
    axes[2].axvline(5.2, color='brown', linestyle='--', linewidth=2)
    axes[2].axvline(np.median(l5_initial_radii), color='red', linestyle='-',
                    linewidth=2, label=f'Median: {np.median(l5_initial_radii):.1f} AU')
axes[2].set_xlabel('Initial Distance (AU)')
axes[2].set_ylabel('Count')
axes[2].set_title(f'L5 Trojan Sources (n={len(l5_initial_radii)})')
axes[2].legend()

plt.tight_layout()
plt.savefig(f'results/png/{config.name}_sources.png', dpi=150)
plt.show()

print(f'Saved: results/png/{config.name}_sources.png')

In [ ]:
# Print source statistics
print('=' * 60)
print('SOURCE REGION STATISTICS')
print('=' * 60)

if len(l4_initial_radii) > 0:
    print(f"L4 Trojans originated from:")
    print(f"  Range: {l4_initial_radii.min():.2f} - {l4_initial_radii.max():.2f} AU")
    print(f"  Median: {np.median(l4_initial_radii):.2f} AU")
    print(f"  Mean: {np.mean(l4_initial_radii):.2f} AU")
    print(f"  Std: {np.std(l4_initial_radii):.2f} AU")
    
print()

if len(l5_initial_radii) > 0:
    print(f"L5 Trojans originated from:")
    print(f"  Range: {l5_initial_radii.min():.2f} - {l5_initial_radii.max():.2f} AU")
    print(f"  Median: {np.median(l5_initial_radii):.2f} AU")
    print(f"  Mean: {np.mean(l5_initial_radii):.2f} AU")
    print(f"  Std: {np.std(l5_initial_radii):.2f} AU")

print('=' * 60)
print('\nUse these source regions to guide particle injection for migration simulations.')

## 6. Final Snapshot

In [ ]:
# Plot final snapshot
fig = plot_snapshot(
    snapshots_massive[-1],
    snapshots_planetesimals[-1],
    times[-1],
    show_trojan_zones=True,
    xlim=(-40, 40),
    ylim=(-40, 40)
)
fig.savefig(f'{config.name}_final.png', dpi=150)
print(f'Saved: {config.name}_final.png')